In [1]:
from charm.toolbox.pairinggroup import PairingGroup, G1,G2,GT, ZR, pair as e,extract_key
from hashlib import sha256, md5
from util1 import obj2bytes, read_word_as_bytes, bxor
import numpy as np  
import sys
import pickle
import socket
import import_ipynb
from Hash_Util import *
import copy
from OdxtUtil import *
class ODXTClientV2:
    def __init__(self, addr):
        self.group = PairingGroup('MNT159')
        self.RList = []
        self.U_Mapping = None
        ###################
        self.ODXT_Param ={} 
       
        self.addr = addr 
        
        

    def opConj(self, op):
        if(op == 'add'):
            return 'del'
        if(op == 'del'):
            return 'add'

    def Setup(self, R):
        
        #初始化新的权限密文R
        ############################################################################################
        p = 69445180235231407255137142482031499329548634082242122837872648805446522657159
        g = 65537
        #生成R对应的私钥sk
        Kt = gen_key_F(H(extract_key(R)))
        Kx = gen_key_F(H_0(extract_key(R)))
        Ky = gen_key_F(H_1(extract_key(R)))
        Kz = gen_key_F(H_2(extract_key(R)))
        #初始化关键词索引表、计数器、交叉索引表
        UpdateCnt, Tset, XSet = dict(), dict(), dict()
        sk, st = (Kt, Kx, Ky, Kz), UpdateCnt
        EDB = (Tset, XSet)
        upCnt = 0
        param = [sk , st , p , g , upCnt]
        self.ODXT_Param[extract_key(R)] = param
        EX_R = extract_key(R)
        conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn.connect(self.addr)
        conn.send(pickle.dumps((0, (EDB, p , EX_R,st))))
        data1 = b""
        while True:
            packet = conn.recv(4096)
            if packet is not None:
                data1 += packet
            if packet is None or len(packet) < 4096:
                break
        data = pickle.loads(data1)
        print(self.ODXT_Param )
        if(data == (1,)):
             print("Setup completed")
        conn.close()

    def Update(self,ES_R, op: str, id_w_tuple):
        #获取对应的sk、计数器等参数
        sk = self.ODXT_Param[ES_R][0]
        st = self.ODXT_Param[ES_R][1]
        p = self.ODXT_Param[ES_R][2]
        g = self.ODXT_Param[ES_R][3]
        upCnt = self.ODXT_Param[ES_R][4]
        upCnt += 1
        self.ODXT_Param[ES_R][4] = upCnt
        id, w = id_w_tuple
        Kt, Kx, Ky, Kz = sk
        if(not w in st):
            st[w] = 0
        st[w] += 1
        w_wc = str(w)+str(st[w])
        #根据公式计算出addr、val、α等参数
        addr = prf_F(Kt, (w_wc+str(0)).encode())
        b1 = (str(op)+str(id)).encode()
        b2 = prf_F(Kt, (w_wc+str(1)).encode())
        b3 = (str(self.opConj(op))+str(id)).encode()
        val = bytes_XOR(b1, b2)
        A0 = prf_Fp(Ky, b1, p, g)
        A = int.from_bytes(A0, 'little')
        A_inv = mul_inv(A, p-1)
        A1 = prf_Fp(Ky, b3, p, g)
        A_p = int.from_bytes(A1, 'little')
        B0 = prf_Fp(Kz, (w_wc).encode(), p, g)
        B = int.from_bytes(B0, 'little')
        B_inv = mul_inv(B, p-1)
        C0 = prf_Fp(Kx, str(w).encode(), p, g)
        C = int.from_bytes(C0, 'little')
        α = (A*B_inv)
        beta = (A_inv*A_p)
        xtag = pow(g, C*A, p)
        self.ODXT_Param[ES_R][1] = st
        st_s = {}
        for key in st.keys():
            st_s[H_3(key.encode())] = st[key]
        
        
        conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn.connect(self.addr)
        conn.send(pickle.dumps((1, (addr, val, (α, beta), xtag, upCnt,ES_R,st_s))))
        data1 = b""
        while True:
            packet = conn.recv(4096)
            if packet is not None:
                data1 += packet
            if packet is None or len(packet) < 4096:
                break
        data = pickle.loads(data1)
        if(data == (1,)):
            print("Update completed")
        conn.close()

    def Search(self, q , r,st):
        p = 69445180235231407255137142482031499329548634082242122837872648805446522657159
        g = 65537
        n = len(q)
        Kt = gen_key_F(H(extract_key(r)))
        Kx = gen_key_F(H_0(extract_key(r)))
        Ky = gen_key_F(H_1(extract_key(r)))
        Kz = gen_key_F(H_2(extract_key(r)))
        MAXINT = sys.maxsize
        w1_uc = MAXINT
        w1 = ""
        for x in q:
            if H_3(x.encode()) in st and st[H_3(x.encode())] < w1_uc:
                w1 = x
                w1_uc = st[H_3(x.encode())]
        stokenlist = []
        xtokenlists = []
        #对每个关键词生成对应的token
        if( H_3(w1.encode()) in st):
            for j in range(w1_uc):
                #根据函数F和Fp计算对应参数addr
                saddr_j = prf_F(
                    Kt, (str(w1)+str(j+1)+str(0)).encode())
                stokenlist.append(saddr_j)
                xtl = []
                B0 = prf_Fp(
                    Kz, (str(w1)+str(j+1)).encode(), p, g)
                B = int.from_bytes(B0, 'little')
                for i in range(n):
                    if(q[i] != w1):
                        A0 = prf_Fp(
                            Kx, (str(q[i])).encode(), p, g)
                        A = int.from_bytes(A0, 'little')
                        xtoken = pow(g, A*B, p)
                        xtl.append(xtoken)
                random.shuffle(xtl)
                xtokenlists.append(xtl)
        res = (stokenlist, xtokenlists,extract_key(r))
        conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn.connect(self.addr)
        conn.send(pickle.dumps((2, res)))

        #
        # SERVER WORK
        #
        data1 = b""
        while True:
            packet = conn.recv(4096)
            if packet is not None:
                data1 += packet
            if packet is None or len(packet) < 4096:
                break
        resp_tup = pickle.loads(data1)
        #通过服务器检索后的文件需要进行去重
        sEOpList = resp_tup[0]
        IdList = []
        for l in sEOpList:
            j, sval, cnt_i, cnt_j = l
            X0 = prf_F(Kt, (str(w1)+str(j+1)+str(1)).encode())
            op_id = bytes_XOR(sval, X0)
            op_id = op_id.decode().rstrip('\x00')
            if(op_id[:3] == 'add' and cnt_i == n and cnt_j == 0):
                IdList.append(int(op_id[3:]))
            elif(op_id[:3] == 'del' and cnt_i > 0 and int(op_id[3:]) in IdList):
                IdList.remove(int(op_id[3:]))
        print(list(set(IdList)))
        conn.close()
        return list(set(IdList))

    def close_server(self):
        conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn.connect(self.addr)
        conn.send(pickle.dumps(("q",)))
        conn.close()
    def getAllNeed(self,):
            conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            conn.connect(self.addr)
            conn.send(pickle.dumps((3,)))
            data1 = b""
            while True:
                packet = conn.recv(4096)
                if packet is not None:
                    data1 += packet
                if packet is None or len(packet) < 4096:
                    break
            data = pickle.loads(data1)
            conn.close()
            return data
    def getKeyGen(self , MSK , S):
            conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            conn.connect(self.addr)
            conn.send(pickle.dumps((1,(MSK , S))))
            data1 = b""
            while True:
                packet = conn.recv(4096)
                if packet is not None:
                    data1 += packet
                if packet is None or len(packet) < 4096:
                    break
            data = pickle.loads(data1)
            conn.close()
            return data

 
    def func_p(self,A,i):
        x = A[i]
        return self.U_Mapping[x]   
    # 添加新的关键词时，向CSP保存对应CT文件到CT列表中
    def EncryptionAU(self,PK , A , R, MSK):
        x = MSK[0]
        g = PK[3]
        AVK = MSK[2]
        g2 = PK[10]
        s = group.random(ZR)
        rows = len(A)
        CT_1 = []
        CT_2 = []
        CT_4 = []
        CT_1s = []
        CT_2s = []
        CT_4s = []
        #根据公式计算出权限密文CT
        for i in range(rows):
            CT_1_i = g2 ** (s * H(self.func_p(A,i).encode()))
            CT_1.append(CT_1_i)
            CT_1s.append(self.group.serialize(CT_1_i))
            CT_2_i = g2 ** (s * AVK[self.func_p(A,i)] * x)
            CT_2.append(CT_2_i)
            CT_2s.append(self.group.serialize(CT_2_i))
            CT_4_i = A[i]
            CT_4.append(CT_4_i)
            CT_4s.append(CT_4_i)
        CT_3 = g2 ** x
        CT_3s = self.group.serialize(CT_3)
        C_0 = R * (e(g,g2) ** (x ** 2)) 
        C_0s = self.group.serialize(C_0)
        return {'CT':[C_0s,CT_1s,CT_2s,CT_3s,CT_4s]}
    #将CT发送到CSP中保存
    def SendCTToCsp(self , CT):
            conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            conn.connect(self.addr)
            conn.send(pickle.dumps((3,(CT))))
            data1 = b""
            while True:
                packet = conn.recv(4096)
                if packet is not None:
                    data1 += packet
                if packet is None or len(packet) < 4096:
                    break
            data = pickle.loads(data1)
            conn.close()
            if(data == (1,)):
                print("Send CT completed")
            
    def VerifySearchAU(self,CT,SK):
        #SK : [K_1s, K_2s,K_3s] 
        #CT : [C_0s,CT_1s,CT_2s,CT_3s,CT_4s]
        #验证用户权限是否被撤销
        if(len(CT[1]) == len(SK[1])):
            flag = 0
            for i in range(len(CT[1])): 
                for k2 in SK[1]:
                        if(e(SK[0],CT[1][i]) == e(k2,CT[2][i])):
                                flag = flag+1
            if(flag == len(CT[1])) :
                R = CT[0] / ((e(SK[0] , CT[3])) ** SK[2])
                return R
            else :
                return 0

        else:
            return 0
    def getCTListFromCSP(self):
        conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn.connect(self.addr)
        conn.send(pickle.dumps((4,)))
        data1 = b""
        while True:
            packet = conn.recv(4096)
            if packet is not None:
                data1 += packet
            if packet is None or len(packet) < 4096:
                break
        data = pickle.loads(data1) #CT_S = []  序列化后的列表
        conn.close()
        return data
    def GetStByR(self,R):
        conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn.connect(self.addr)
        conn.send(pickle.dumps((5,(extract_key(R)))))
        data1 = b""
        while True:
            packet = conn.recv(4096)
            if packet is not None:
                data1 += packet
            if packet is None or len(packet) < 4096:
                break
        data = pickle.loads(data1)  #CT_S = []  序列化后的列表
        conn.close()
        return data
    def getRL_ListFromTA(self):
        conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn.connect(self.addr)
        conn.send(pickle.dumps((4,)))
        data1 = b""
        while True:
            packet = conn.recv(4096)
            if packet is not None:
                data1 += packet
            if packet is None or len(packet) < 4096:
                break
        data = pickle.loads(data1) 
        conn.close()
        return data
        
        
    
    

     
               
               
               
                
           

        
HOST = 'localhost'
TA_PORT = 50098
CSP_PORT = 50068
group = PairingGroup('MNT159')

if __name__ == "__main__":
    #HOST = sys.argv[1]
    #PORT = int(sys.argv[2])
    # HOST = 'localhost'
    # PORT = 50060
    PK = None
    MSK = None
    Mapping = None
    SK = None
    Attr_R = {}
    R_Local = []
    Attr = []
    
    client_obj = ODXTClientV2((HOST, TA_PORT))
    client2_obj = ODXTClientV2((HOST, CSP_PORT))
    print('欢迎进入Client操作界面用odxt可搜索加密系统')
    while True:
        option = input('请输入选项前面的数字：1. 初始化Client ;2.初始化密钥 3.添加内容  ;4.删除内容 5.查找内容 ；6.更新私钥 ； exit退出')
        if option == '1':
            #将PK   MSK ， U_mapping拿过来 ，获得私钥 
            Coll = client_obj.getAllNeed()  # PK , MSK , U_Mapping {'PK':pk , 'MSK' : msk , 'U_mapping' : mapping}
            PK = Coll['PK']
            PK[3] = group.deserialize(PK[3])
            PK[7] = group.deserialize(PK[7])
            PK[8] = group.deserialize(PK[8])
            for key in PK[9].keys():
                PK[9][key] = group.deserialize(PK[9][key])
            PK[10] = group.deserialize(PK[10])    
            MSK = Coll['MSK']
            MSK[0] = group.deserialize(MSK[0])
            MSK[1] = group.deserialize(MSK[1])
            for key in MSK[2].keys():
                MSK[2][key] = group.deserialize(MSK[2][key])
            Mapping = Coll['U_mapping']
            client_obj.U_Mapping = Coll['U_mapping']
            client2_obj.U_Mapping = Coll['U_mapping']
            print(PK)
            print(MSK)
            print(Mapping)
            
            
        elif option == '2':
            #初始化私钥SK
            S = []
            temp = input('输入用户属性文件名 ' )
            with open(temp, 'r') as f:
                S = f.readlines()
                S = [line.strip() for line in S]
            
            for value in S:
                 Attr.append(get_key(Mapping,value)[0])
            
            #序列化MSK，同时不影响原先MSK
            MSK_s = []
            MSK_s.append(group.serialize(MSK[0]))
            MSK_s.append(group.serialize(MSK[1]))
            MSK_s_3 = {}
            for key in MSK[2].keys():
                MSK_s_3[key] = group.serialize(MSK[2][key])
            MSK_s.append(MSK_s_3)        
               
            MSK_s[0] = group.serialize(MSK[0])
            MSK_s[1] = group.serialize(MSK[1])
            #获取私钥
            data = client_obj.getKeyGen(MSK_s , S)
            print(MSK)
            if(data['SK'] == 0) :
                print('主密钥MSK失效，请重新初始化获取')
                continue;
            
            #{'SK' : [K_1s, K_2s,K_3s] }
            print(data)
            SK = data['SK']
            #反序列化私钥信息
            SK[0] = group.deserialize(SK[0])
            SK[2] = group.deserialize(SK[2])
            for i in range(len(SK[1])):
                SK[1][i] = group.deserialize(SK[1][i])
                
                
            print(SK)
            
        elif option == '3' :
            #输入想要输入的数据
            id = int(input('请输入 id>'))
            name = input('请输入name>')
            #输入访问控制策略
            S = []
            temp = input('输入可访问的用户属性文件名 ' )
            with open(temp, 'r') as f:
                S = f.readlines()
                S = [line.strip() for line in S]
            #根据Mapping形成访问列表A Mapping = {1111:"apple" , 2222 : "banana",1234 : "puple" ,2314 : "student" , 4567 : "teacher"}
            Real_attrs = []
            for o in S:
                if (o in Mapping.values() and not(o in Real_attrs) ):
                    Real_attrs.append(o)
            key = [] ## key列表为过滤完不在U_Mapping和重复属性后的Mapping键列表
            for value in Real_attrs:
                 key.append(get_key(Mapping,value)[0])
            #调用函数生成CT 之前 需要先判断是否有必要生成新的R
            #若为新的R则将对称密钥存储在 R_Local = []中  并调用SetUp对对应R对应的内容进行初始化
            #更新属性和R的对应表
            key_tuple = tuple(key)
            R = None
            if( Attr_R == {}) :
                R = group.random(GT)
                R_value = extract_key(R) # H(R)  映射值
                Attr_R[key_tuple] = R
                R_Local.append(R_value)
                client2_obj.Setup(R)
            else:
                for attr in Attr_R.keys():
                    if ((len(key_tuple)  == 1 and attr == key_tuple[0]) or (set(attr) == set(key_tuple))):
                        R = Attr_R[attr]
                        break
                        
            if(R == None):
                R = group.random(GT)
                R_value = extract_key(R) # H(R)  映射值 List内包含的值
                Attr_R[key_tuple] = R
                R_Local.append(R_value)
                client2_obj.Setup(R)
            #调用函数生成CT
            client2_obj.RList = R_Local
            CT = client2_obj.EncryptionAU(PK , key , R, MSK)
            
            #传输到CSP中进行保存
            client2_obj.SendCTToCsp(CT)
            #加密R
            EX_R = extract_key(R)
            #调用Update函数，根据对称密钥值将对应数据上传存储
            client2_obj.Update(EX_R,'add', (id, name))
            
            
        elif option == '4':
            id = int(input('请输入 id>'))
            name = input('请输入name >')
            for EX_R in R_Local: 
                client2_obj.Update(EX_R ,'del',(id, name))
            print("Del Completed")
            
            
            
            
            
                
        elif option == '5':
            print("查找内容")
             #输入name进行查询
            shuru = input('请输入name ，不同关键字间用“ ”隔开>')
            name = [n for n in shuru.split(" ")]
            #获取云端CT列表
            CT_s_List = client2_obj.getCTListFromCSP()  #序列化后的CT_List
            print(CT_s_List)
            CT_S = []  #反序列化后的CT列表
            for CT in CT_s_List:
                CT_s = []
                CT_s_0 = group.deserialize(CT[0])
                CT_1s = []
                CT_2s = []
                CT_4s = []
                for i in range(len(CT[1])):
                    CT_1s.append(group.deserialize(CT[1][i]))
                    CT_2s.append(group.deserialize(CT[2][i]))
                    CT_4s.append(CT[4][i])
                CT_s_3 = group.deserialize(CT[3])
                CT_s = [CT_s_0,CT_1s,CT_2s,CT_s_3,CT_4s]
                CT_S.append(CT_s)
            print(CT_S)
            
            #验证属性是否满足对应访问控制 ， 返回对应的R列表
            R_confirm = []  ##保存的R
            for ct in CT_S:
                data = client2_obj.VerifySearchAU(ct,SK)
                if not(data == 0):
                    R_confirm.append(data)
                    break;
            if R_confirm == []:
                print("没有匹配的权限")
                continue;
           #对于每个R 执行下面函数进行搜索
           #之前需要拿到每个r对应的st
            Result = []
            for r in R_confirm:
                st = client2_obj.GetStByR(r) 
                data = client2_obj.Search(name , r , st) 
                Result.append(data)
            print(Result)
                
        elif option == '6':
            ##更新PK ， MSK
            Coll = client_obj.getAllNeed()  # PK , MSK , U_Mapping {'PK':pk , 'MSK' : msk , 'U_mapping' : mapping}
            PK = Coll['PK']
            PK[3] = group.deserialize(PK[3])
            PK[7] = group.deserialize(PK[7])
            PK[8] = group.deserialize(PK[8])
            for key in PK[9].keys():
                PK[9][key] = group.deserialize(PK[9][key])
            PK[10] = group.deserialize(PK[10])    
            MSK = Coll['MSK']
            MSK[0] = group.deserialize(MSK[0])
            MSK[1] = group.deserialize(MSK[1])
            for key in MSK[2].keys():
                MSK[2][key] = group.deserialize(MSK[2][key])
            #从TA获取RL列表
            data = client_obj.getRL_ListFromTA()
            RL = {}
            for sita in data.keys():
                RL[sita] = group.deserialize(data[sita])
            g = PK[3]
            
            for v in RL.keys():
                for i in range(len(Attr)):
                    if(Attr[i] ==  v):
                        SK[1][i] = SK[1][i] ** (1 / RL[v])
        
            print(SK)   
            print("撤销后的SK更新完成")
            
            
                
            
            
        elif option == 'exit':
            print('退出！')
            break
        else:
            print('输入无效，请重新输入')           

importing Jupyter notebook from Hash_Util.ipynb
欢迎进入Client操作界面用odxt可搜索加密系统
请输入选项前面的数字：1. 初始化Client ;2.初始化密钥 3.添加内容  ;4.删除内容 5.查找内容 ；6.更新私钥 ； exit退出1
[1, 3, 625852803282871856053922297323874661378036491717, [67670034117901088302471891500900068128301842325, 351411600503115005307864910245446233119316954842], <function H at 0x7f37f16e1560>, <function H_0 at 0x7f37f16e15f0>, <function H_1 at 0x7f37f16e1710>, [599062962005684170429660645395367583282310437526, 549282965177291177610161870034314203135689880352], [239634237453467663243594615414103837383051502954, 220523098720200581368656578823990312238243991338], {'podhale': [597086521258196297732497918950127446248186208660, 359227681411940002367655281043121046597779724613], 'accomplish': [47858607267870047258264487161858802047939626432, 404738541047273733240660469248340283134794982411], 'galois': [259041333908186461659112944293422274411019992635, 37136506250203674663447652595781596084918085785], 'bradenham': [63260291935804333995143933631536811

请输入选项前面的数字：1. 初始化Client ;2.初始化密钥 3.添加内容  ;4.删除内容 5.查找内容 ；6.更新私钥 ； exit退出3
请输入 id>123456
请输入name>apple
输入可访问的用户属性文件名 5.txt
{b'\xd5(\xa9-\xc3\x14\xe8J\x9c\xd4)\x05:nb\xf9\x12h\x8b\xe4\x07$\x0e\xcf\x99\xfb\\KmmH\xb0': [(b'\x18d\x9dc\x0fi-\x1aU\x97_v\r>zf\x8aE\x17\xd6]\xb3\xc6\xbc%\x7f\x86l\xf9*q]', b"`t\x87r\x08j\xc4\x1e\xbb\x94\xcfa9\x83\x8f'\xc1}\xbb\xcc/\x04\xba\xce\xce|\xcaw\xd9\xc7\xd6\xbf", b'\x7f\xb5wrY>\x89\x90\x9f\xdc\xe8\xaa\xa8\xa3\x98\xb4\xd9\x7f\xf8H\xa1\xb4"f\xc8q\x03\xd1\xd8kC\x11', b'\xf7S\x99\xbf\x14\x1a\x82~\x9a\xad\x99[cx\r\xbb\xa2\xeb\x8d^\x1d\x17\xe1\xd6\x8a\x99\xb8\xdc\x7f*\x0e\xf0'), {}, 69445180235231407255137142482031499329548634082242122837872648805446522657159, 65537, 0]}
Setup completed
Send CT completed
Update completed
请输入选项前面的数字：1. 初始化Client ;2.初始化密钥 3.添加内容  ;4.删除内容 5.查找内容 ；6.更新私钥 ； exit退出5
查找内容
请输入name ，不同关键字间用“ ”隔开>apple
[[b'3:IQvAzCRPyziaFf5WcwO4JCHFkuspQh3i/yJwZUr5DyFFCWkSSjfN4AJ6NbynMrFdUfYwCmz9D/6xB94YTQd3OD3G4VNgqDSDWqgE4TK1wRpIe+elPngCCiVQ2IhRxVxHFFw

请输入选项前面的数字：1. 初始化Client ;2.初始化密钥 3.添加内容  ;4.删除内容 5.查找内容 ；6.更新私钥 ； exit退出6
[[468447284355359729190681200885084775273393936460, 525877263366195368373717138557274296440017489729], [[159818192199453001037975147802946208773899802087, 600799672265882992751514010363886828219184873801], [291321412737567184520666610884210124218491950208, 515624971118833780269332091295703229092217788808], [443301812533483814534748923789294239544294067808, 501154448479394936449141804793361895489289402163], [221322129710170517032414462986135783104581362855, 45014534954255886591309894335291603945672791446], [85136678133963276525068514494482442585243167326, 312935333012640545049387470093069999211643968876]], 201542096917565529051476549286233209071171914356]
撤销后的SK更新完成
请输入选项前面的数字：1. 初始化Client ;2.初始化密钥 3.添加内容  ;4.删除内容 5.查找内容 ；6.更新私钥 ； exit退出5
查找内容
请输入name ，不同关键字间用“ ”隔开>apple
[[b'3:IQvAzCRPyziaFf5WcwO4JCHFkuspQh3i/yJwZUr5DyFFCWkSSjfN4AJ6NbynMrFdUfYwCmz9D/6xB94YTQd3OD3G4VNgqDSDWqgE4TK1wRpIe+elPngCCiVQ2IhRxVxHFFw/OgwqVYQ

KeyboardInterrupt: Interrupted by user